In [2]:
#README This file is using for testing some parameter only


import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import os 




/home/thanawit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/thanawit/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
#initialte the variable

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation  = True
num_predictions = 20 
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
#downloading the dataset

(x_train , y_train) , (x_test , y_test ) = cifar10.load_data()

In [5]:
#convert class to binary class

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test =  keras.utils.to_categorical(y_test, num_classes)

In [9]:
x_train.shape[1:]

(32, 32, 3)

In [10]:
#Initialize Layer

model = Sequential()
#Conv2D Filter = filter size, kernel size, stride , padding , format , dilation rate , activation , biased, initializer, bias initilizaer, more 

#Block 1 
model.add(Conv2D( 32 , (3,3) , padding='same',input_shape = x_train.shape[1:]  ))
model.add(Activation('sigmoid'))
model.add(Conv2D(32 , (3,3)))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Block 2 
model.add(Conv2D(64, (3,3) , padding='same'))
model.add(Activation('sigmoid'))
model.add(Conv2D(64 , (3,3)))
model.add(Activation('sigmoid'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Connecting it to Classification
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('sigmoid'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))



In [11]:

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ lr_reducer, lr_scheduler]



In [12]:
#initiate training lost 
opt = keras.optimizers.rmsprop(lr_schedule(0), decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#meaning all the result
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

Learning rate:  0.001


In [13]:
#Subtract mean to make it more normalize
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

In [14]:
#Decaying LR

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ lr_reducer, lr_scheduler]




datagen = ImageDataGenerator(
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    )
    

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)



#train model 
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks, steps_per_epoch = 50000/batch_size)
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Learning rate:  0.001
Epoch 1/100
1563/1562 [==============================] - 13s - loss: 1.6175 - acc: 0.4546 - val_loss: 1.1530 - val_acc: 0.6101
Learning rate:  0.001
Epoch 2/100
1563/1562 [==============================] - 12s - loss: 1.2096 - acc: 0.5872 - val_loss: 0.9182 - val_acc: 0.6744
Learning rate:  0.001
Epoch 3/100
1563/1562 [==============================] - 12s - loss: 1.0877 - acc: 0.6287 - val_loss: 0.8966 - val_acc: 0.6848
Learning rate:  0.001
Epoch 4/100
1563/1562 [==============================] - 12s - loss: 1.0012 - acc: 0.6552 - val_loss: 0.8087 - val_acc: 0.7265
Learning rate:  0.001
Epoch 5/100
1563/1562 [==============================] - 12s - loss: 0.9311 - acc: 0.6826 - val_loss: 0.7243 - val_acc: 0.7470
Learning rate:  0.001
Epoch 6/100
1563/1562 [==============================] - 12s - loss: 0.8764 - acc: 0.7002 - val_loss: 0.7719 - val_acc: 0.7481
Learning rate:  0.001
Epoch 7/100
1563/1562 [==============================] - 12s - loss: 0.8224 - acc: 0

In [16]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

 9600/10000 [===========================>..] - ETA: 0sTest loss: 0.43081979093551637
Test accuracy: 0.8688


In [15]:
model.load()

AttributeError: 'Sequential' object has no attribute 'load'